In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import r2_score

import pickle

#from eli5.sklearn import PermutationImportance

In [2]:
glacier_ids = np.load("../results/misc/glacier_ids_valid.npy")

In [3]:
glacier_ids.shape

(28074,)

In [4]:
tsl_path = "../results/tsl_csv/"
meteo_path = "../data/FIT_forcing/meteo/"

In [5]:
def read_tsl(rgi_id, path=tsl_path):
    
    tsl = pd.read_csv(f"{tsl_path}{rgi_id}.csv", index_col=0, parse_dates=True)
    
    return tsl

def read_meteo(rgi_id, path=meteo_path):
    
    meteo = pd.read_hdf(f"{meteo_path}{rgi_id}.h5")
    
    return meteo

In [6]:
def create_features(dataframe, back_to=12):
    
    # convert circular wind_dir_mean 
    # to two components of cos() and sin()
    # source: https://stats.stackexchange.com/questions/336613/
    # regression-using-circular-variable-hour-from-023-as-predictor
    
    # copy for safety
    df = dataframe.copy()
    
    # create cos() and sin() components
    df["wind_dir_mean_cos"] = np.cos(np.deg2rad(df["wind_dir_mean"]))
    df["wind_dir_mean_sin"] = np.sin(np.deg2rad(df["wind_dir_mean"]))
    
    # drop "wind_dir_mean"
    df = df.drop(["wind_dir_mean"], axis=1)
    
    # make shifts and rolling means
    cols = df.columns
    for col in cols:
        for shift in range(1, back_to+1, 1):
            df["{}-{}".format(col, shift)] = df[col].shift(shift).values
        for rol in range(1, back_to+1, 1):
            df["{}rol-{}".format(col, rol)] = df[col].rolling(window=rol).mean().values
    
    # delete NaNs
    df = df.dropna()
       
    return df

In [7]:
def datasets_construction(rgi_id, freq, subset_jjas):
    
    # get raw TSL measurements
    tsl = read_tsl(rgi_id)
    
    # resample to specific frequency
    tsl_resample = tsl.resample(freq).mean()
    
    # get raw ERA5-Land forcing
    meteo = read_meteo(rgi_id)
    
    # resample to specific frequency
    meteo_resample = pd.DataFrame({'t2m_min': meteo['t2m_min'].resample(freq).min(), 
                                   't2m_max': meteo['t2m_max'].resample(freq).max(), 
                                   't2m_mean': meteo['t2m_mean'].resample(freq).mean(), 
                                   'tp': meteo['tp'].resample(freq).sum(), 
                                   'sf': meteo['sf'].resample(freq).sum(),
                                   'ssrd': meteo['ssrd'].resample(freq).sum(), 
                                   'strd': meteo['strd_mean'].resample(freq).sum(),
                                   'wind_max': meteo['wind_max'].resample(freq).max(), 
                                   'wind_mean': meteo['wind_mean'].resample(freq).mean(), 
                                   'wind_dir_mean': meteo['wind_dir_mean'].resample(freq).mean(),
                                   'tcc': meteo['tcc'].resample(freq).mean()})
    
    # enrich meteo features
    if freq == "M":
        meteo_enrich = create_features(meteo_resample, back_to=12)
    elif freq == "W":
        meteo_enrich = create_features(meteo_resample, back_to=48) #12 months back considering 4 weeks in each month
    
    
    # meteo for the entire period
    # for model evaluation
    meteo_full = meteo_enrich.dropna()
    
    # merge datasets
    dataset = pd.concat([tsl_resample, meteo_enrich], axis=1)
    
    # drop NaNs
    dataset = dataset.dropna()
    
    if subset_jjas:
        dataset = dataset[(dataset.index.month == 6) | (dataset.index.month == 7) | 
                          (dataset.index.month == 8) | (dataset.index.month == 9)]
    
    return dataset, meteo_full

In [8]:
def tiny_prep(df):
       
    df_X = df.drop(["TSL_normalized"], axis=1)
    df_y = df["TSL_normalized"]
    
    return df_X, df_y

In [9]:
def calculate_importances(imp_instance):
    
    cols = imp_instance.columns.tolist()
    
    # temperature-based
    t2m_min_cols = [i for i in cols if "t2m_min" in i]
    t2m_max_cols = [i for i in cols if "t2m_max" in i]
    t2m_mean_cols = [i for i in cols if "t2m_mean" in i]
    
    # precipitation-based
    tp_cols = [i for i in cols if "tp" in i]
    sf_cols = [i for i in cols if "sf" in i]
    
    # surface solar radiation downwards
    ssrd_cols = [i for i in cols if "ssrd" in i]
    
    # surface thermal radiation downwards
    strd_cols = [i for i in cols if "strd" in i]
    
    # wind-based
    wind_max_cols = [i for i in cols if "wind_max" in i]
    wind_mean_cols = [i for i in cols if "wind_mean" in i]
    wind_dir_mean_cols = [i for i in cols if "wind_dir_mean" in i]
    
    # total cloud cover
    tcc_cols = [i for i in cols if "tcc" in i]
    
    var_importances = []
    
    for var in [t2m_min_cols, 
                t2m_max_cols, 
                t2m_mean_cols, 
                tp_cols, 
                sf_cols,
                ssrd_cols, 
                strd_cols, 
                wind_max_cols, 
                wind_mean_cols, 
                wind_dir_mean_cols,
                tcc_cols]:
        
        var_importances.append(imp_instance[var].sum(axis=0).sum())
        
    var_importances = np.array(var_importances)
    
    var_importances = var_importances / var_importances.sum()
    
    return var_importances

In [10]:
def ML_me(rgi_id, freq, subset_jjas):
    
    # data: features and target
    df, meteo_full = datasets_construction(rgi_id, freq, subset_jjas)
    
    features_df, target_df = tiny_prep(df)
    features, target = features_df.values, target_df.values 
    
    # model: RandomForest for regression
    # parsimonious model with low complexity
    # n_estimators is better to be multiple of available CPU threads
    model = RandomForestRegressor(random_state=76, n_estimators=8, n_jobs=-1)
    
    # holders for each step predictions/observations
    obs = []
    prd = []
    
    # leave-one-out cross-validation
    loo = LeaveOneOut()
    
    # predictions for whole interval
    full_preds = []
    
    # feature importances holder
    importances = []
    
    # misc prefixes
    if freq == "M":
        freq_prefix = "monthly"
    elif freq == "W":
        freq_prefix = "weekly"
    
    if subset_jjas:
        subset_prefix = "JJAS"
    else:
        subset_prefix = "full"
    
    # training loop
    for train, test in loo.split(target): 
        
        # split data on train/test
        X_train, y_train, X_test, y_test = features[train], target[train], features[test], target[test]
        
        # fir data on train
        model.fit(X_train, y_train)
        
        # calculate test prediction
        y_pred = model.predict(X_test)
        
        # calculate prediction for the entire dataset
        full_pred = model.predict(meteo_full.values)
        
        # add test and predicted values to holders
        obs.append(y_test[0])
        prd.append(y_pred[0])
        full_preds.append(full_pred)
                
        # fit eli5 to calculate permutation imporatnces
        #pi = PermutationImportance(estimator=model, cv="prefit", n_iter=1).fit(X_train, y_train)
        
        # get feature importances from eli5
        #fi = pi.feature_importances_
        
        # get feature importances from native model
        fi = model.feature_importances_
        
        # convert importances to dataframe
        fi_df = pd.DataFrame({0: fi}, index=features_df.columns).T
        
        # collect
        importances.append(fi_df)
        
        # save model instance
        # models are large -- not feasible
        #pickle.dump(model, open(f"../results/tsl_ML/{freq_prefix}_{subset_prefix}/trained_models/{rgi_id}_{test}.pkl", 'wb'))
                  
    
    # grab predictions together
    obs = np.array(obs)
    prd = np.array(prd)
    
    # calculate r2 LOO score
    loo_score = r2_score(obs, prd)
    
    # grab loo obs and preds together
    loo_sim = pd.DataFrame({"obs": obs, "sim": prd}, index=features_df.index)
    # save loo predicitions
    loo_sim.to_csv(f"../results/tsl_ML/{freq_prefix}_{subset_prefix}/tsl_simulated/{rgi_id}_loo.csv", 
                   compression="gzip")
    
    # postprocessing of entire predictions
    full_preds = np.array(full_preds).reshape(len(target), -1)
    ensemble_mean = full_preds.mean(axis=0)
    tsl_for_meteo_full = pd.DataFrame({"TSL_sim": ensemble_mean}, index=meteo_full.index)
    # save ensemble mean for the entire meteo ts
    tsl_for_meteo_full.to_csv(f"../results/tsl_ML/{freq_prefix}_{subset_prefix}/tsl_simulated/{rgi_id}_ens.csv", 
                              compression="gzip")
    
    # get importances together
    all_importances = pd.concat(importances, axis=0, ignore_index=True)
    
    # calculate relative importances
    rel_importances = calculate_importances(all_importances)
    
    #return loo_sim, loo_score, rel_importances, tsl_for_meteo_full
    return loo_score, rel_importances

In [11]:
glacier_ids.shape

(28074,)

In [12]:
%%time
list_loo_scores = []
list_importances = []

broken = []

for num, idx in enumerate(glacier_ids):
        
    try:
        individual_loo_score, individual_importances = ML_me(idx, "M", True)
    
    except:
        print(idx, "Warning")
        broken.append(idx)
        
    list_loo_scores.append(individual_loo_score)
    list_importances.append(individual_importances)
    
    print(num, 
          idx, 
          np.round(individual_loo_score, 2),  
          np.argmax(individual_importances))

0 RGI60-13.00014 -0.42 6
1 RGI60-13.00015 0.03 1
2 RGI60-13.00017 -0.27 7
3 RGI60-13.00021 0.03 0
4 RGI60-13.00023 0.35 0
5 RGI60-13.00026 -0.48 9
6 RGI60-13.00031 -0.2 8
7 RGI60-13.00035 -0.16 4
8 RGI60-13.00040 0.43 5
9 RGI60-13.00044 0.22 1
10 RGI60-13.00046 0.13 1
11 RGI60-13.00049 0.14 6
12 RGI60-13.00052 0.08 2
13 RGI60-13.00055 -0.08 5
14 RGI60-13.00061 -0.37 7
15 RGI60-13.00062 0.11 1
16 RGI60-15.04831 -0.45 5
17 RGI60-15.04832 -0.06 8
18 RGI60-15.04833 -0.14 9
19 RGI60-15.04834 0.01 0
20 RGI60-15.04835 0.13 8
21 RGI60-15.04838 -0.55 8
22 RGI60-15.04839 -0.33 8
23 RGI60-15.04840 -0.09 9
24 RGI60-15.04843 0.15 5
25 RGI60-15.04844 0.07 1
26 RGI60-15.04846 -0.16 1
27 RGI60-15.04847 -0.03 8
28 RGI60-15.04850 -0.18 4
29 RGI60-15.04851 0.41 1
30 RGI60-15.04852 -0.1 5
31 RGI60-15.04853 0.5 8
32 RGI60-15.04854 -0.22 0
33 RGI60-15.04855 -0.2 6
34 RGI60-15.04856 -0.33 7
35 RGI60-15.05153 0.21 6
36 RGI60-15.05154 0.11 9
37 RGI60-15.05158 0.19 1
38 RGI60-15.05159 -0.28 9
39 RGI60-15.05160 

313 RGI60-15.05012 -0.27 9
314 RGI60-15.05013 -0.18 10
315 RGI60-15.05014 -0.01 9
316 RGI60-15.05022 -0.16 9
317 RGI60-15.05024 -0.61 3
318 RGI60-15.05027 -0.52 9
319 RGI60-15.05028 -0.29 9
320 RGI60-15.05029 -0.01 9
321 RGI60-15.05031 -0.13 9
322 RGI60-15.05032 -0.08 1
323 RGI60-15.05033 -0.17 9
324 RGI60-15.05038 0.18 9
325 RGI60-14.08230 0.05 1
326 RGI60-14.08232 0.38 1
327 RGI60-14.08234 0.44 2
328 RGI60-14.08238 -0.07 3
329 RGI60-14.08240 0.16 2
330 RGI60-14.08242 -0.15 6
331 RGI60-14.08244 -0.33 9
332 RGI60-14.08245 0.21 0
333 RGI60-14.08248 -0.04 5
334 RGI60-14.08253 0.33 5
335 RGI60-14.08258 0.43 2
336 RGI60-14.08259 0.19 2
337 RGI60-14.08260 0.17 5
338 RGI60-14.08262 0.33 2
339 RGI60-14.08263 0.25 1
340 RGI60-14.08264 0.21 2
341 RGI60-14.08267 -0.1 2
342 RGI60-14.19218 0.19 9
343 RGI60-14.19224 0.29 9
344 RGI60-14.19225 0.08 4
345 RGI60-14.19229 -0.38 7
346 RGI60-14.19230 0.24 0
347 RGI60-14.19263 0.31 6
348 RGI60-14.19269 0.13 1
349 RGI60-14.19270 0.25 6
350 RGI60-14.19271 0.

623 RGI60-14.19066 -0.05 2
624 RGI60-14.19075 0.23 1
625 RGI60-14.19077 0.37 1
626 RGI60-14.19078 0.39 4
627 RGI60-14.19079 0.1 3
628 RGI60-14.19080 0.19 8
629 RGI60-14.19084 -0.17 9
630 RGI60-14.19087 0.42 3
631 RGI60-15.05832 0.05 2
632 RGI60-15.05840 -0.36 9
633 RGI60-15.05841 -0.21 1
634 RGI60-15.05843 -0.28 5
635 RGI60-15.05844 -0.39 9
636 RGI60-15.05845 -0.14 10
637 RGI60-15.05850 0.32 9
638 RGI60-15.05853 0.1 6
639 RGI60-15.05859 -0.03 4
640 RGI60-15.05866 0.08 0
641 RGI60-15.05869 0.02 4
642 RGI60-15.05871 0.13 9
643 RGI60-15.05872 -0.06 4
644 RGI60-15.05879 0.17 7
645 RGI60-15.05881 -0.17 9
646 RGI60-15.05882 0.29 2
647 RGI60-15.05885 0.25 2
648 RGI60-15.05888 0.43 0
649 RGI60-15.05893 -0.39 3
650 RGI60-15.05894 0.6 2
651 RGI60-15.05895 -0.86 8
652 RGI60-15.05896 -0.47 9
653 RGI60-15.05899 0.5 2
654 RGI60-15.05907 0.27 1
655 RGI60-14.26268 -0.05 2
656 RGI60-14.26269 0.03 10
657 RGI60-14.26280 0.19 2
658 RGI60-14.26288 -0.05 9
659 RGI60-14.26295 -0.03 9
660 RGI60-14.26308 0.12 

933 RGI60-14.08496 -0.02 0
934 RGI60-14.08497 -0.29 5
935 RGI60-14.08499 -0.05 10
936 RGI60-14.08502 -0.39 8
937 RGI60-14.08504 0.05 1
938 RGI60-14.08505 0.09 4
939 RGI60-14.08509 -0.26 8
940 RGI60-14.08510 -0.03 1
941 RGI60-14.08512 -0.05 5
942 RGI60-13.05379 -0.19 9
943 RGI60-13.05381 -0.14 4
944 RGI60-13.05384 -0.33 5
945 RGI60-13.05385 -0.1 4
946 RGI60-13.05389 -0.32 9
947 RGI60-13.05392 -0.12 4
948 RGI60-13.05393 0.04 3
949 RGI60-13.05395 -0.25 7
950 RGI60-13.05396 -0.45 9
951 RGI60-13.05402 -0.4 7
952 RGI60-13.05403 -0.05 9
953 RGI60-13.05405 -0.28 9
954 RGI60-13.05409 -0.22 9
955 RGI60-13.05410 -0.46 9
956 RGI60-13.05416 -0.16 9
957 RGI60-13.05427 -0.24 9
958 RGI60-13.05432 -0.2 4
959 RGI60-13.05435 -0.33 3
960 RGI60-13.11378 -0.3 9
961 RGI60-13.11387 0.29 1
962 RGI60-13.11412 -0.53 4
963 RGI60-13.11414 -0.36 7
964 RGI60-13.11415 0.12 0
965 RGI60-13.11416 -0.22 7
966 RGI60-13.11419 -0.0 8
967 RGI60-13.11425 0.06 6
968 RGI60-13.11426 -0.35 9
969 RGI60-13.11428 0.09 1
970 RGI60-13

1232 RGI60-15.05616 -0.02 2
1233 RGI60-15.05617 0.2 9
1234 RGI60-15.05619 0.11 1
1235 RGI60-15.05622 -0.62 9
1236 RGI60-15.05623 0.19 9
1237 RGI60-15.05628 -0.69 9
1238 RGI60-15.05631 -0.23 4
1239 RGI60-15.05634 -0.49 9
1240 RGI60-15.05638 -0.54 4
1241 RGI60-15.05639 -0.44 7
1242 RGI60-15.05645 -0.44 8
1243 RGI60-13.11066 0.11 8
1244 RGI60-13.11075 0.2 8
1245 RGI60-13.11086 0.12 6
1246 RGI60-13.11104 -0.01 0
1247 RGI60-13.11115 -0.05 3
1248 RGI60-13.11116 0.33 4
1249 RGI60-13.11120 -0.01 9
1250 RGI60-13.11121 -0.19 9
1251 RGI60-13.11123 -0.01 9
1252 RGI60-13.11136 0.12 4
1253 RGI60-13.11159 -0.78 9
1254 RGI60-13.11161 0.37 6
1255 RGI60-13.11163 0.08 0
1256 RGI60-13.11166 0.28 0
1257 RGI60-13.11169 -0.14 6
1258 RGI60-13.11170 0.16 0
1259 RGI60-13.11176 0.26 7
1260 RGI60-13.11191 0.16 2
1261 RGI60-13.11205 0.41 0
1262 RGI60-14.19726 -0.29 7
1263 RGI60-14.19728 0.36 2
1264 RGI60-14.19733 -0.15 6
1265 RGI60-14.19735 -0.01 3
1266 RGI60-14.19736 -0.38 8
1267 RGI60-14.19744 -0.16 9
1268 RGI60

...

26638 RGI60-13.47350 -0.12 9
26639 RGI60-13.47353 0.02 1
26640 RGI60-13.47355 -0.03 9
26641 RGI60-13.47356 -0.05 9
26642 RGI60-13.47374 -0.27 9
26643 RGI60-13.47376 0.0 4
26644 RGI60-13.47378 -0.58 9
26645 RGI60-13.47380 -0.13 10
26646 RGI60-13.47381 -0.33 9
26647 RGI60-13.47382 -0.54 10
26648 RGI60-13.47398 0.03 9
26649 RGI60-13.47408 -0.07 0
26650 RGI60-13.47409 0.37 6
26651 RGI60-13.47132 0.03 0
26652 RGI60-13.47134 -0.48 9
26653 RGI60-13.47135 -0.0 9
26654 RGI60-13.47137 0.25 9
26655 RGI60-13.47139 -1.13 1
26656 RGI60-13.47141 -0.21 3
26657 RGI60-13.47150 -0.05 9
26658 RGI60-13.47152 -0.26 9
26659 RGI60-13.47153 -0.31 4
26660 RGI60-13.47154 0.04 9
26661 RGI60-13.47159 -0.08 0
26662 RGI60-13.47167 -1.3 1
26663 RGI60-13.47168 -0.34 7
26664 RGI60-13.47169 -0.21 7
26665 RGI60-13.47180 -0.1 2
26666 RGI60-13.47182 -0.06 0
26667 RGI60-13.47184 0.09 6
26668 RGI60-13.47187 0.01 7
26669 RGI60-13.47194 0.12 2
26670 RGI60-13.47200 -0.22 0
26671 RGI60-13.47202 0.16 0
26672 RGI60-13.47207 0.13 9

26927 RGI60-13.50514 -0.34 7
26928 RGI60-13.50515 -0.13 9
26929 RGI60-13.50516 -0.1 10
26930 RGI60-13.50518 -0.29 7
26931 RGI60-13.50521 0.08 9
26932 RGI60-13.50530 -0.24 0
26933 RGI60-13.50531 -0.23 9
26934 RGI60-13.50536 -0.46 5
26935 RGI60-13.50538 -0.04 2
26936 RGI60-13.50545 -0.42 7
26937 RGI60-13.50551 -0.03 9
26938 RGI60-13.50564 -0.24 9
26939 RGI60-13.50566 0.22 5
26940 RGI60-13.50575 0.18 10
26941 RGI60-13.50585 -0.04 5
26942 RGI60-13.50598 -0.19 9
26943 RGI60-13.50608 -0.15 9
26944 RGI60-13.50610 -0.35 7
26945 RGI60-13.50611 -0.4 9
26946 RGI60-13.50614 -0.17 9
26947 RGI60-13.50630 -0.19 6
26948 RGI60-13.49985 -0.1 2
26949 RGI60-13.49986 -0.32 5
26950 RGI60-13.49995 -0.42 4
26951 RGI60-13.50015 0.11 9
26952 RGI60-13.50020 -0.19 7
26953 RGI60-13.50023 -0.31 8
26954 RGI60-13.50026 -0.21 5
26955 RGI60-13.50030 0.03 9
26956 RGI60-13.50038 -0.15 2
26957 RGI60-13.50045 -0.2 8
26958 RGI60-13.50051 -0.25 9
26959 RGI60-13.50053 -0.28 9
26960 RGI60-13.50054 -0.25 4
26961 RGI60-13.50068 

27213 RGI60-13.50960 -0.53 0
27214 RGI60-13.50766 0.05 9
27215 RGI60-13.50767 -0.1 4
27216 RGI60-13.50769 -0.13 9
27217 RGI60-13.50773 -0.37 9
27218 RGI60-13.50775 -0.26 8
27219 RGI60-13.50776 0.02 9
27220 RGI60-13.50778 -0.14 9
27221 RGI60-13.50780 -0.13 9
27222 RGI60-13.50782 -0.33 9
27223 RGI60-13.50783 -0.18 9
27224 RGI60-13.50786 -0.44 5
27225 RGI60-13.50787 -0.06 9
27226 RGI60-13.50792 -0.29 9
27227 RGI60-13.50794 -0.11 6
27228 RGI60-13.50795 -0.18 1
27229 RGI60-13.50796 0.28 4
27230 RGI60-13.51079 -0.21 8
27231 RGI60-13.51084 -0.48 8
27232 RGI60-13.51086 -0.36 2
27233 RGI60-13.51088 -0.36 5
27234 RGI60-13.51095 -0.09 9
27235 RGI60-13.51098 -0.25 4
27236 RGI60-13.51099 -0.07 8
27237 RGI60-13.51102 -0.04 9
27238 RGI60-13.51104 -0.13 4
27239 RGI60-13.51110 -0.39 2
27240 RGI60-13.51117 -0.08 4
27241 RGI60-13.51118 -0.03 7
27242 RGI60-13.51123 -0.35 9
27243 RGI60-13.51126 -0.11 9
27244 RGI60-13.51128 0.07 5
27245 RGI60-13.51134 -0.17 2
27246 RGI60-13.51136 0.17 2
27247 RGI60-13.51139

27503 RGI60-13.51853 0.07 6
27504 RGI60-13.51924 -0.0 5
27505 RGI60-13.51929 0.09 0
27506 RGI60-13.51944 0.09 3
27507 RGI60-13.51956 -0.07 9
27508 RGI60-13.51958 -0.33 10
27509 RGI60-13.51969 0.26 0
27510 RGI60-13.51970 0.14 2
27511 RGI60-13.51972 0.18 0
27512 RGI60-13.51974 0.11 0
27513 RGI60-13.51978 -0.11 0
27514 RGI60-13.51985 0.16 6
27515 RGI60-13.51986 0.29 2
27516 RGI60-13.51990 0.14 4
27517 RGI60-13.51996 0.01 9
27518 RGI60-13.52000 0.03 9
27519 RGI60-13.52003 -0.29 7
27520 RGI60-13.52004 -0.49 4
27521 RGI60-13.52006 -0.24 9
27522 RGI60-13.52011 -0.01 8
27523 RGI60-13.52020 -0.03 4
27524 RGI60-13.52023 -0.18 9
27525 RGI60-13.52024 0.18 0
27526 RGI60-13.52033 0.37 2
27527 RGI60-13.52034 0.05 4
27528 RGI60-13.52035 0.04 9
27529 RGI60-13.52036 0.39 0
27530 RGI60-13.52037 -0.04 5
27531 RGI60-13.52040 -0.23 9
27532 RGI60-13.52042 0.09 0
27533 RGI60-13.52049 0.04 0
27534 RGI60-13.52062 0.27 0
27535 RGI60-13.52064 0.11 6
27536 RGI60-13.52065 -0.03 4
27537 RGI60-13.52088 0.32 0
27538 R

27792 RGI60-13.53554 0.1 1
27793 RGI60-13.53556 0.31 2
27794 RGI60-13.53557 0.06 6
27795 RGI60-13.53558 0.24 2
27796 RGI60-13.53561 -0.21 9
27797 RGI60-13.53565 -0.02 0
27798 RGI60-13.53568 0.18 1
27799 RGI60-13.53575 -0.01 1
27800 RGI60-13.53578 0.17 1
27801 RGI60-13.53583 -0.01 1
27802 RGI60-13.53595 0.03 5
27803 RGI60-13.53598 0.16 10
27804 RGI60-13.53599 0.37 6
27805 RGI60-13.53484 0.03 2
27806 RGI60-13.53485 0.22 0
27807 RGI60-13.53486 0.33 0
27808 RGI60-13.53487 0.08 1
27809 RGI60-13.53492 -0.03 2
27810 RGI60-13.53496 -0.09 2
27811 RGI60-13.53497 0.09 2
27812 RGI60-13.53498 0.19 2
27813 RGI60-13.53507 0.25 2
27814 RGI60-13.53510 0.12 2
27815 RGI60-13.53519 0.19 5
27816 RGI60-13.53520 0.05 2
27817 RGI60-13.53522 0.16 2
27818 RGI60-13.53527 0.36 2
27819 RGI60-13.53529 0.25 2
27820 RGI60-13.53531 0.25 1
27821 RGI60-13.53620 0.1 4
27822 RGI60-13.53621 0.39 4
27823 RGI60-13.53622 0.15 4
27824 RGI60-13.53624 0.07 8
27825 RGI60-13.53632 -0.05 1
27826 RGI60-13.53636 0.32 0
27827 RGI60-13

In [13]:
scores_loo_arr = np.array(list_loo_scores)
importances_arr = np.array(list_importances)

In [14]:
scores_loo_arr.shape, importances_arr.shape

((28074,), (28074, 11))

In [15]:
broken

['RGI60-15.06327',
 'RGI60-14.21668',
 'RGI60-13.18300',
 'RGI60-15.00505',
 'RGI60-15.00521',
 'RGI60-15.00535',
 'RGI60-15.00215',
 'RGI60-15.01029',
 'RGI60-15.01065',
 'RGI60-15.01082',
 'RGI60-15.01146',
 'RGI60-15.01199',
 'RGI60-15.01271',
 'RGI60-15.01431',
 'RGI60-15.01483',
 'RGI60-15.01484',
 'RGI60-15.01592',
 'RGI60-15.01496',
 'RGI60-15.02118',
 'RGI60-15.02565',
 'RGI60-15.02786',
 'RGI60-15.02996',
 'RGI60-15.03739',
 'RGI60-15.04045',
 'RGI60-15.03945',
 'RGI60-15.03968',
 'RGI60-15.04770',
 'RGI60-15.09349',
 'RGI60-15.09273',
 'RGI60-15.09293',
 'RGI60-15.09536',
 'RGI60-15.09493',
 'RGI60-15.09873',
 'RGI60-15.09877',
 'RGI60-15.09912',
 'RGI60-15.11846',
 'RGI60-15.12169',
 'RGI60-15.12219',
 'RGI60-15.12300',
 'RGI60-15.12370',
 'RGI60-15.12395',
 'RGI60-15.12465',
 'RGI60-15.12471',
 'RGI60-15.12476',
 'RGI60-15.12542',
 'RGI60-14.03702',
 'RGI60-14.04402',
 'RGI60-13.00950',
 'RGI60-13.01159',
 'RGI60-13.01187',
 'RGI60-13.01190',
 'RGI60-13.01451',
 'RGI60-13.0

In [16]:
res = pd.DataFrame({"id": glacier_ids, 
                    "t2m_min":  importances_arr[:, 0], 
                    't2m_max':  importances_arr[:, 1],
                    't2m_mean': importances_arr[:, 2],
                    'tp':       importances_arr[:, 3], 
                    'sf':       importances_arr[:, 4],
                    'ssrd':     importances_arr[:, 5],
                    'strd':     importances_arr[:, 6],
                    'wind_max': importances_arr[:, 7],
                    'wind_mean':importances_arr[:, 8], 
                    "wind_dir_mean": importances_arr[:, 9],
                    'tcc':      importances_arr[:, 10],
                    "score_loo":scores_loo_arr})

In [17]:
res.describe()

,t2m_min,t2m_max,t2m_mean,tp,sf,ssrd,strd,wind_max,wind_mean,wind_dir_mean,tcc,score_loo
count,28027.000000,28027.000000,28027.000000,28027.000000,28027.000000,28027.000000,28027.000000,28027.000000,28027.000000,28027.000000,28027.000000,28074.000000
mean,0.086520,0.092471,0.100335,0.065603,0.108866,0.078274,0.065899,0.087466,0.083441,0.148243,0.082883,-0.053787
std,0.066025,0.071728,0.081916,0.047148,0.076165,0.065715,0.052581,0.057315,0.056603,0.079112,0.054731,0.294505
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000
25%,0.041945,0.042375,0.042605,0.034187,0.055795,0.034976,0.031500,0.048782,0.045105,0.091937,0.045636,-0.209083
50%,0.068419,0.071855,0.075788,0.052739,0.088482,0.058416,0.050996,0.072712,0.068394,0.130577,0.069161,-0.031704
75%,0.110916,0.120069,0.132181,0.082777,0.139912,0.099497,0.083079,0.109898,0.105582,0.185172,0.104944,0.135938
max,0.708303,0.673609,0.650766,0.564131,0.758263,0.708947,0.772137,0.662102,0.653164,0.667305,0.720749,1.000000


In [18]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28074 entries, 0 to 28073
Data columns (total 13 columns):
id               28074 non-null object
t2m_min          28027 non-null float64
t2m_max          28027 non-null float64
t2m_mean         28027 non-null float64
tp               28027 non-null float64
sf               28027 non-null float64
ssrd             28027 non-null float64
strd             28027 non-null float64
wind_max         28027 non-null float64
wind_mean        28027 non-null float64
wind_dir_mean    28027 non-null float64
tcc              28027 non-null float64
score_loo        28074 non-null float64
dtypes: float64(12), object(1)
memory usage: 2.8+ MB


In [19]:
res[["id", "tcc", "score_loo"]][res["tcc"].isnull() == True]

,id,tcc,score_loo
1310,RGI60-14.20079,NaN,-3.0
2515,RGI60-14.21690,NaN,1.0
5143,RGI60-13.42309,NaN,-3.0
6081,RGI60-15.00886,NaN,-3.0
6144,RGI60-15.00504,NaN,-3.0
6145,RGI60-15.00505,NaN,-3.0
6148,RGI60-15.00524,NaN,-3.0
6230,RGI60-15.00209,NaN,-3.0
6231,RGI60-15.00211,NaN,-3.0
6232,RGI60-15.00215,NaN,-3.0


In [20]:
res.to_csv("../results/tsl_ML/monthly_JJAS/misc/monthly_JJAS_drivers.csv")